
# Project 1 — Parts c)–f)
This notebook solves **c, d, e, f** using the provided modules in this repo:
`data.py`, `grad.py`, `metrics.py`, `models.py`, `plots.py`, `resampling.py`, `utils.py`.


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import sys, pathlib

# Project-1 folder (parent of this Notebooks/ folder)
proj_dir = pathlib.Path.cwd().parent
if str(proj_dir) not in sys.path:
    sys.path.insert(0, str(proj_dir))

# Figures folder
fig_dir = proj_dir / "Figures"
fig_dir.mkdir(parents=True, exist_ok=True)

from Code.data import make_data, build_features, split_and_scale

# --- Project-wide experiment config (used in ALL notebooks) ---

# Data settings
N_SAMPLES   = 300     # number of samples
NOISE_SD    = 0.3     # noise standard deviation
SEED_DATA   = 42      # data generation
SEED_SPLIT  = 42      # train/test split (split_and_scale uses this)
DEG_MAX     = 15      # max polynomial degree for features
P_FIXED     = 15      # fixed polynomial degree for regularization experiments
TEST_SIZE   = 0.20    # train/test split size


# Regularization grids
LAM_GRID_RIDGE = np.logspace(-6, 1, 40)  # wider range for ridge
LAM_GRID_LASSO = np.logspace(-6, 0, 60)  # less range for LASSO

# Optimizer settings
ETA_FULL   = 5e-3     # full-batch GD steps
ITERS_FULL = 5000     # full-batch GD iterations
LAM_RIDGE  = 0.0203   # Taken from ridge experiments (02_Ridge_lambda.ipynb) best lambda for degree 15
BETA = 0.3    # momentum parameter
B1 = 0.9    # Adam parameter
B2 = 0.999  # Adam parameter
EPS = 1e-8  # Adam parameter
RHO = 0.99  # Adadelta parameter

# Mini-batch SGD
EPOCHS_MB    = 25     # passes over data
BATCH_SIZE   = 64     # minibatch size
ETA_MB_OLS   = 1e-2   # for OLS, can be larger
ETA_MB_RIDGE = 1e-2   # for ridge, need to tune
SEED_MB      = 42     # minibatch shuffle

# --- Generate data explicitly---
x, y = make_data(n=N_SAMPLES, noise_sd=NOISE_SD, seed=SEED_DATA)
X_full = build_features(x, degree=DEG_MAX, include_bias=False)


X_tr_s, X_te_s, y_tr_c, y_te, scaler, y_mean = split_and_scale(X_full, y, test_size=TEST_SIZE, random_state=SEED_SPLIT)


## c) Gradient Descent for OLS and Ridge (using `grad.py`)

We first use the plain gradient descent method.


In [5]:
from Code.metrics import mse, r2
from Code.models import predict_centered
from Code.grad import gd, grad_ridge, grad_ols

# Standardized design matrix, p is number of features
p = X_tr_s.shape[1]  

# OLS via gradient descent
theta_ols = gd(X_tr_s, y_tr_c, eta=ETA_FULL, iters=ITERS_FULL, theta0=np.zeros(p), lam=None) # lam=None → OLS
yhat_te_ols = predict_centered(X_te_s, theta_ols, y_mean) # add back y_mean
mse_ols = mse(y_te, yhat_te_ols) 
r2_ols  = r2(y_te, yhat_te_ols) 

# Ridge via gradient descent (λ given)
theta_ridge = gd(X_tr_s, y_tr_c, eta=ETA_FULL, iters=ITERS_FULL, theta0=np.zeros(p), lam=LAM_RIDGE, n_factor=True) # lam=float → Ridge
yhat_te_ridge = predict_centered(X_te_s, theta_ridge, y_mean) # add back y_mean
mse_ridge = mse(y_te, yhat_te_ridge) 
r2_ridge  = r2(y_te, yhat_te_ridge)

print(f"OLS  — Test MSE: {mse_ols:.5f}, R²: {r2_ols:.4f}")
print(f"Ridge (λ={LAM_RIDGE}) — Test MSE: {mse_ridge:.5f}, R²: {r2_ridge:.4f}")

OLS  — Test MSE: 0.15016, R²: 0.2461
Ridge (λ=0.0203) — Test MSE: 0.15017, R²: 0.2461



## d) Optimizers: Momentum, AdaGrad, RMSProp, Adam

We keep the same objectives but change the update rule. We'll use both Ridge and OLS. 


In [6]:
# --- Optimizer (no SGD) ---
def run_optimizer(X, y, lam=LAM_RIDGE, eta=ETA_FULL, iters=ITERS_FULL, optimizer="gd",
                  beta=BETA, eps=EPS, rho=RHO, b1=B1, b2=B2, seed=SEED_DATA):
    """
    Full-batch optimizers for OLS/Ridge:
      gd        : plain gradient descent (full batch)
      momentum  : GD with momentum
      adagrad   : adaptive per-parameter steps
      rmsprop   : exponential moving avg of squared grads
      adam      : momentum + rmsprop with bias correction
    No stochastic SGD here as we'll do that later.
    """
    n, p = X.shape
    rng = np.random.default_rng(SEED_DATA)  # kept for parity; not used in full-batch
    theta = np.zeros(p, float)
    v = np.zeros_like(theta)  # momentum buffer
    s = np.zeros_like(theta)  # second-moment buffer
    m = np.zeros_like(theta)  # first-moment buffer
    t = 0

    for _ in range(ITERS_FULL):
        g = grad_ridge(X, y, theta, LAM_RIDGE) if lam is not None else grad_ols(X, y, theta)
        t += 1

        if optimizer in ("gd",):                     # plain full-batch GD
            theta -= ETA_FULL * g
        elif optimizer == "momentum":
            v = BETA * v + (1 - BETA) * g
            theta -= ETA_FULL * v
        elif optimizer == "adagrad":
            s += g * g
            theta -= (ETA_FULL / (np.sqrt(s) + EPS)) * g
        elif optimizer == "rmsprop":
            s = RHO * s + (1 - RHO) * (g * g)
            theta -= (ETA_FULL / (np.sqrt(s) + EPS)) * g
        elif optimizer == "adam":
            m = B1 * m + (1 - B1) * g
            s = B2 * s + (1 - B2) * (g * g)
            m_hat = m / (1 - B1**t)
            s_hat = s / (1 - B2**t)
            theta -= ETA_FULL * m_hat / (np.sqrt(s_hat) + EPS)
        else:
            raise ValueError(f"Unknown optimizer: {optimizer}")
    return theta

# --- Run comparisons ---
opts = ["gd", "momentum", "adagrad", "rmsprop", "adam"]
results = {}

for opt in opts:
    th = run_optimizer(X_tr_s, y_tr_c, lam=LAM_RIDGE, eta=ETA_FULL, iters=ITERS_FULL, optimizer=opt)
    yhat = predict_centered(X_te_s, th, y_mean)
    results[opt] = (mse(y_te, yhat), r2(y_te, yhat))

print(f" FULL-BATCH (RIDGE, λ={LAM_RIDGE})")

# Pretty print
for k, (mse_v, r2_v) in results.items():
    print(f"{k:8s}  MSE={mse_v:.5f}  R²={r2_v:.4f}")

for opt in opts:
    th = run_optimizer(X_tr_s, y_tr_c, lam=None, eta=ETA_FULL, iters=ITERS_FULL, optimizer=opt)
    yhat = predict_centered(X_te_s, th, y_mean)
    results[opt] = (mse(y_te, yhat), r2(y_te, yhat))

print(" FULL-BATCH (OLS, λ=None)")

# Pretty print
for k, (mse_v, r2_v) in results.items():
    print(f"{k:8s}  MSE={mse_v:.5f}  R²={r2_v:.4f}")


 FULL-BATCH (RIDGE, λ=0.0203)
gd        MSE=0.15017  R²=0.2461
momentum  MSE=0.15017  R²=0.2461
adagrad   MSE=0.15077  R²=0.2431
rmsprop   MSE=0.13550  R²=0.3197
adam      MSE=0.13055  R²=0.3446
 FULL-BATCH (OLS, λ=None)
gd        MSE=0.15016  R²=0.2461
momentum  MSE=0.15017  R²=0.2461
adagrad   MSE=0.15075  R²=0.2431
rmsprop   MSE=0.13517  R²=0.3214
adam      MSE=0.12921  R²=0.3513



## e) LASSO 



In [7]:
# --- LASSO (scikit-learn) on the same split ---
from sklearn.linear_model import Lasso
from Code.metrics import mse, r2

# choose a reasonable λ grid for standardized features


Xtr, Xte = X_tr_s[:, :p], X_te_s[:, :p]  # fixed degree p
mses, r2s, norms = [], [], []

# (optional) warm-start: going from large -> small lambdas is faster
for lam in LAM_GRID_LASSO:
    model = Lasso(alpha=lam,
                  fit_intercept=False,    # we centered y_tr -> intercept handled by y_mean
                  max_iter=ITERS_FULL,
                  tol=1e-4,
                  random_state=SEED_DATA,
                  selection="cyclic")
    model.fit(Xtr, y_tr_c)
    theta = model.coef_.ravel()

    # add back the training mean for predictions
    yhat = Xte @ theta + y_mean

    mses.append(mse(y_te, yhat))
    r2s.append(r2(y_te, yhat))
    norms.append(float(np.linalg.norm(theta)))

mses = np.asarray(mses); r2s = np.asarray(r2s); norms = np.asarray(norms)

# Plot test MSE and R² vs λ

""" fig, ax1 = plt.subplots(figsize=(9,5))
ax1.plot(lambdas, mses, 'o-', label='Test MSE')
ax1.set_xscale('log'); ax1.set_xlabel('λ'); ax1.set_ylabel('MSE (test)'); ax1.grid(True, ls='--', alpha=0.4)

ax2 = ax1.twinx()
ax2.plot(lambdas, r2s, 's--', color='C2', label='Test R²')
ax2.set_ylabel('R² (test)')

ax1.set_title(f'LASSO vs λ (degree={p}), noise=0.3')
ax1.legend(loc='best')
plt.tight_layout(); plt.show()

# (optional) coefficient norm diagnostic
plt.figure(figsize=(8,4))
plt.plot(lambdas, norms, 'x-')
plt.xscale('log'); plt.xlabel('λ'); plt.ylabel(r'||θ||₂'); plt.grid(True, ls='--', alpha=0.4)
plt.title('LASSO coefficient size vs λ, noise=0.3, degree=15'); plt.tight_layout(); plt.show() """

best_idx = int(np.argmin(mses))
print("\nBest by MSE:")
print(f"λ* = {LAM_GRID_LASSO[best_idx]:.2e}  |  MSE* = {mses[best_idx]:.6f}  |  R²* = {r2s[best_idx]:.4f}")


/Users/selmabeateovland/Documents/Fys-stk4155/Oppgaver og prosjekt/Project1/FYS-STK4155/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.061e+01, tolerance: 4.234e-03
  model = cd_fast.enet_coordinate_descent(
/Users/selmabeateovland/Documents/Fys-stk4155/Oppgaver og prosjekt/Project1/FYS-STK4155/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.058e+01, tolerance: 4.234e-03
  model = cd_fast.enet_coordinate_descent(
/Users/selmabeateovland/Documents/Fys-stk4155/Oppgaver og prosjekt/Project1/FYS-STK4155/.venv/lib/python3.12/site-packages/sklearn/linea


Best by MSE:
λ* = 1.00e-06  |  MSE* = 0.129401  |  R²* = 0.3503


/Users/selmabeateovland/Documents/Fys-stk4155/Oppgaver og prosjekt/Project1/FYS-STK4155/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.927e-02, tolerance: 4.234e-03
  model = cd_fast.enet_coordinate_descent(
/Users/selmabeateovland/Documents/Fys-stk4155/Oppgaver og prosjekt/Project1/FYS-STK4155/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.362e-02, tolerance: 4.234e-03
  model = cd_fast.enet_coordinate_descent(
/Users/selmabeateovland/Documents/Fys-stk4155/Oppgaver og prosjekt/Project1/FYS-STK4155/.venv/lib/python3.12/site-packages/sklearn/linea

In [13]:
""" #Lasso via (sub)gradient descent with fixed learning rate """

def lasso_gd(X, y, theta, eta, lmbda, Niterations):
 
    #LASSO via plain (sub)gradient descent with fixed learning rate.

   
    n = X.shape[0]  # number of samples
    for iter in range(Niterations):
        # 1) OLS gradient component: (2/n) * X^T (X theta - y)
        residual = X @ theta - y
        OLS_gradient_component = (2.0 / n) * (X.T @ residual)

        # 2) L1 gradient component (subgradient): lambda * sign(theta)
        # (note: at theta_i = 0 the subgradient is any value in [-1, 1]; using sign(0)=0 is a common practical choice)
        L1_gradient_component = lmbda * np.sign(theta)

        # 3) Total gradient
        total_gradient = OLS_gradient_component + L1_gradient_component

        # 4) Parameter update with fixed learning rate
        theta -= eta * total_gradient

    return theta 




In [9]:
# --- Compare LASSO: scikit vs. GD at λ=2.73e-05 (best lambda value from tests above)---

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso

lam = 2.73e-05  

# 1) scikit-learn Lasso 
lasso_sk = Lasso(alpha=lam, fit_intercept=False, max_iter=20000, tol=1e-4, random_state=42)
lasso_sk.fit(X_tr_s, y_tr_c)
theta_sk = lasso_sk.coef_.ravel()
yhat_sk  = X_te_s @ theta_sk + y_mean
mse_sk   = mse(y_te, yhat_sk)
r2_sk    = r2(y_te, yhat_sk)

# 2) Your GD Lasso at same λ
p = X_tr_s.shape[1]
theta0 = np.zeros(p)
eta = 5e-3
Niterations = 3000  # bump iterations a bit for a fairer comparison
theta_gd = lasso_gd(X_tr_s, y_tr_c, theta=theta0, eta=eta, lmbda=lam, Niterations=Niterations)
yhat_gd  = X_te_s @ theta_gd + y_mean
mse_gd   = mse(y_te, yhat_gd)
r2_gd    = r2(y_te, yhat_gd)

print(f"LASSO (scikit, λ={lam}) — Test MSE: {mse_sk:.6f}, R²: {r2_sk:.4f}")
print(f"LASSO (GD    , λ={lam}) — Test MSE: {mse_gd:.6f}, R²: {r2_gd:.4f}")

# ---------- Plot 1: overlay fitted curves on a dense grid ----------

""" xgrid = np.linspace(-1, 1, 400)
Xgrid_full = build_features(xgrid, degree=X_tr_s.shape[1], include_bias=False)  # degree = p features
Xgrid_s    = scaler.transform(Xgrid_full)[:, :p]

# True function for reference
fgrid = 1/(1+25*xgrid**2)

ygrid_sk = Xgrid_s @ theta_sk + y_mean
ygrid_gd = Xgrid_s @ theta_gd + y_mean

plt.figure(figsize=(9,5))
plt.plot(xgrid, fgrid, label='True f(x)', lw=2)
plt.plot(xgrid, ygrid_sk, label=f'LASSO scikit (λ={lam})')
plt.plot(xgrid, ygrid_gd, label=f'LASSO GD (λ={lam})', ls='--')
plt.title('LASSO fits on dense grid (degree = p)')
plt.xlabel('x'); plt.ylabel('y')
plt.grid(True, ls='--', alpha=0.4); plt.legend(); plt.tight_layout()
plt.show() """




/Users/selmabeateovland/Documents/Fys-stk4155/Oppgaver og prosjekt/Project1/FYS-STK4155/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.929e+00, tolerance: 4.234e-03
  model = cd_fast.enet_coordinate_descent(


LASSO (scikit, λ=2.73e-05) — Test MSE: 0.128600, R²: 0.3544
LASSO (GD    , λ=2.73e-05) — Test MSE: 0.149394, R²: 0.2500


" xgrid = np.linspace(-1, 1, 400)\nXgrid_full = build_features(xgrid, degree=X_tr_s.shape[1], include_bias=False)  # degree = p features\nXgrid_s    = scaler.transform(Xgrid_full)[:, :p]\n\n# True function for reference\nfgrid = 1/(1+25*xgrid**2)\n\nygrid_sk = Xgrid_s @ theta_sk + y_mean\nygrid_gd = Xgrid_s @ theta_gd + y_mean\n\nplt.figure(figsize=(9,5))\nplt.plot(xgrid, fgrid, label='True f(x)', lw=2)\nplt.plot(xgrid, ygrid_sk, label=f'LASSO scikit (λ={lam})')\nplt.plot(xgrid, ygrid_gd, label=f'LASSO GD (λ={lam})', ls='--')\nplt.title('LASSO fits on dense grid (degree = p)')\nplt.xlabel('x'); plt.ylabel('y')\nplt.grid(True, ls='--', alpha=0.4); plt.legend(); plt.tight_layout()\nplt.show() "


## f) Stochastic Gradient Descent



In [10]:
def sgd_minibatch_sum(
    X, y, lam=1e-3, epochs=40, batch_size=64, eta=1e-2, optimizer="sgd",
    beta=0.9, eps=1e-8, rho=0.99, b1=0.9, b2=0.999, seed=0
):
    """
    Mini-batch SGD (sum form) for OLS/Ridge with per-sample loss:
        c_i(θ) = 1/2 (x_i^T θ - y_i)^2 + (lam/2) ||θ||^2
    Batch gradient (sum form):
        G_B = X_B^T (X_B θ - y_B) + lam * |B| * θ
    Update:
        θ <- θ - η * G_B
    """
    n, p = X.shape
    rng = np.random.default_rng(seed)
    theta = np.zeros(p, float)

    # state buffers (for momentum/Adagrad/RMSProp/Adam)
    v = np.zeros_like(theta)
    s = np.zeros_like(theta)
    m = np.zeros_like(theta)
    t = 0

    for _ in range(epochs):
        idx = rng.permutation(n) # shuffle data each epoch
        for start in range(0, n, batch_size):
            B = idx[start:start+batch_size] # batch indices
            Xb, yb = X[B], y[B]
            nb = Xb.shape[0]

            r = Xb @ theta - yb                            # residuals
            g = (Xb.T @ r) + lam * nb * theta              # SUM-FORM gradient

            t += 1
            if optimizer == "sgd":
                theta -= eta * g
            elif optimizer == "momentum":
                v = beta * v + (1 - beta) * g
                theta -= eta * v
            elif optimizer == "adagrad":
                s += g * g
                theta -= (eta / (np.sqrt(s) + eps)) * g
            elif optimizer == "rmsprop":
                s = rho * s + (1 - rho) * (g * g)
                theta -= (eta / (np.sqrt(s) + eps)) * g
            elif optimizer == "adam":
                m = b1 * m + (1 - b1) * g
                s = b2 * s + (1 - b2) * (g * g)
                m_hat = m / (1 - b1**t)
                s_hat = s / (1 - b2**t)
                theta -= eta * m_hat / (np.sqrt(s_hat) + eps)
            else:
                raise ValueError("Unknown optimizer")
    return theta


lam_val = 2.73e-05
B0, eta0 = 64, 1e-2
B = 128
eta = eta0 * (B0 / B)  # scale eta for sum-form
epochs = int(1.5 * 25)

# full-batch (e.g., Adam)
theta_full = run_optimizer(X_tr_s, y_tr_c, lam=lam_val, eta=eta0, iters=5000, optimizer="adam")

# stochastic (sum form), same optimizer rule
theta_sgd  = sgd_minibatch_sum(X_tr_s, y_tr_c, lam=lam_val, epochs=epochs, batch_size=B, eta=eta, optimizer="adam", seed=42)

yhat_full = predict_centered(X_te_s, theta_full, y_mean)
yhat_sgd  = predict_centered(X_te_s, theta_sgd,  y_mean)

print(f"Full-batch Adam (λ={lam_val}):  MSE={mse(y_te,yhat_full):.6f}, R²={r2(y_te,yhat_full):.4f}")
print(f"SGD (sum form) Adam (λ={lam_val}, B={64}):  MSE={mse(y_te,yhat_sgd):.6f}, R²={r2(y_te,yhat_sgd):.4f}")


Full-batch Adam (λ=2.73e-05):  MSE=0.130551, R²=0.3446
SGD (sum form) Adam (λ=2.73e-05, B=64):  MSE=0.158845, R²=0.2025


In [11]:
def sgd_minibatch(
    X, y, lam=1e-3, epochs=40, batch_size=64, eta=1e-2, optimizer="sgd",
    beta=0.9, eps=1e-8, rho=0.99, b1=0.9, b2=0.999, seed=42
):
    """
    Part f) Mini-batch SGD framework using the SAME update rules as parts c–e.
    Matches the 1/(2n) loss convention: grad = (1/nb) X_b^T (X_b θ - y_b) + lam * θ
    Supports: 'sgd', 'momentum', 'adagrad', 'rmsprop', 'adam'
    """
    n, p = X.shape
    rng = np.random.default_rng(seed)
    theta = np.zeros(p, float)

    # state buffers (shared across optimizers)
    v = np.zeros_like(theta)  # momentum
    s = np.zeros_like(theta)  # second moment (AdaGrad/RMSProp/Adam)
    m = np.zeros_like(theta)  # first moment (Adam)
    t = 0

    for _ in range(epochs):
        idx = rng.permutation(n)
        for start in range(0, n, batch_size):
            b = idx[start:start+batch_size]
            Xb, yb = X[b], y[b]
            nb = Xb.shape[0]

            # gradient for ridge MSE with 1/(2n) convention
            r = Xb @ theta - yb
            g = (Xb.T @ r) / nb + lam * theta

            t += 1
            if optimizer == "sgd":
                theta -= eta * g

            elif optimizer == "momentum":
                v = beta * v + (1 - beta) * g
                theta -= eta * v

            elif optimizer == "adagrad":
                s += g * g
                theta -= (eta / (np.sqrt(s) + eps)) * g

            elif optimizer == "rmsprop":
                s = rho * s + (1 - rho) * (g * g)
                theta -= (eta / (np.sqrt(s) + eps)) * g

            elif optimizer == "adam":
                m = b1 * m + (1 - b1) * g
                s = b2 * s + (1 - b2) * (g * g)
                m_hat = m / (1 - b1**t)
                s_hat = s / (1 - b2**t)
                theta -= eta * m_hat / (np.sqrt(s_hat) + eps)

            else:
                raise ValueError(f"Unknown optimizer: {optimizer}")

    return theta

theta_full = run_optimizer(X_tr_s, y_tr_c, lam=2.73e-05, eta=5e-3, iters=5000, optimizer="adam")
theta_sgd  = sgd_minibatch(X_tr_s, y_tr_c, lam=2.73e-05, epochs=25, batch_size=64, eta=5e-3, optimizer="adam")

yhat_full = predict_centered(X_te_s, theta_full, y_mean)
yhat_sgd  = predict_centered(X_te_s, theta_sgd,  y_mean)
print("Full-batch Adam:", mse(y_te, yhat_full), r2(y_te, yhat_full))
print("Mini-batch Adam:", mse(y_te, yhat_sgd),  r2(y_te, yhat_sgd))


Full-batch Adam: 0.13055141405037554 0.3445609716177649
Mini-batch Adam: 0.156467557704395 0.21444784998240352


In [14]:
import time
import numpy as np
from sklearn.linear_model import Lasso




# ===== Leaderboard (now includes mini-batch rows) =====

# config
optimizers_full = ["gd", "momentum", "adagrad", "rmsprop", "adam"]   # full-batch
optimizers_mb   = ["sgd", "momentum", "adagrad", "rmsprop", "adam"]  # mini-batch
eta_ols_full    = 5e-3
eta_ridge_full  = 5e-3
iters_full      = 5000
lam_ridge       = 1e-3
lam_lasso       = 1e-3

# mini-batch hyperparams
epochs_mb    = 25
batch_size   = 64
eta_mb_ols   = 1e-2
eta_mb_ridge = 1e-2
seed_mb      = 42

Xtr, Xte = X_tr_s, X_te_s  # ensure you've sliced to degree p earlier if needed

results = []

def add_result(name, theta, yhat, t0):
    results.append({
        "method":  name,
        "MSE":     float(mse(y_te, yhat)),
        "R2":      float(r2(y_te, yhat)),
        "||θ||2":  float(np.linalg.norm(theta)),
        "time_s":  time.perf_counter() - t0
    })

# --- OLS: closed form ---
t0 = time.perf_counter()
theta_ols_cf = np.linalg.pinv(Xtr) @ y_tr_c
yhat_ols_cf  = predict_centered(Xte, theta_ols_cf, y_mean)
add_result("OLS closed-form", theta_ols_cf, yhat_ols_cf, t0)

# --- OLS: full-batch optimizers ---
for opt in optimizers_full:
    t0 = time.perf_counter()
    theta = run_optimizer(Xtr, y_tr_c, lam=None, eta=eta_ols_full, iters=iters_full, optimizer=opt)
    yhat  = predict_centered(Xte, theta, y_mean)
    add_result(f"OLS {opt}", theta, yhat, t0)

# --- OLS: mini-batch optimizers ---
for opt in optimizers_mb:
    t0 = time.perf_counter()
    theta = sgd_minibatch(Xtr, y_tr_c, lam=0.0, epochs=epochs_mb, batch_size=batch_size,
                          eta=eta_mb_ols, optimizer=opt, seed=seed_mb)
    yhat  = predict_centered(Xte, theta, y_mean)
    add_result(f"OLS {opt} (mini-batch)", theta, yhat, t0)

# --- Ridge: closed form ---
t0 = time.perf_counter()
theta_ridge_cf = np.linalg.solve(Xtr.T @ Xtr + lam_ridge * np.eye(Xtr.shape[1]), Xtr.T @ y_tr_c)
yhat_ridge_cf  = predict_centered(Xte, theta_ridge_cf, y_mean)
add_result("Ridge closed-form", theta_ridge_cf, yhat_ridge_cf, t0)

# --- Ridge: full-batch optimizers ---
for opt in optimizers_full:
    t0 = time.perf_counter()
    theta = run_optimizer(Xtr, y_tr_c, lam=lam_ridge, eta=eta_ridge_full, iters=iters_full, optimizer=opt)
    yhat  = predict_centered(Xte, theta, y_mean)
    add_result(f"Ridge {opt}", theta, yhat, t0)

# --- Ridge: mini-batch optimizers ---
for opt in optimizers_mb:
    t0 = time.perf_counter()
    theta = sgd_minibatch(Xtr, y_tr_c, lam=lam_ridge, epochs=epochs_mb, batch_size=batch_size,
                          eta=eta_mb_ridge, optimizer=opt, seed=seed_mb)
    yhat  = predict_centered(Xte, theta, y_mean)
    add_result(f"Ridge {opt} (mini-batch)", theta, yhat, t0)

# --- Lasso: scikit (coordinate descent) ---
t0 = time.perf_counter()
lasso = Lasso(alpha=lam_lasso, fit_intercept=False, max_iter=20000, tol=1e-4, random_state=42)
lasso.fit(Xtr, y_tr_c)
theta_lasso = lasso.coef_.ravel()
yhat_lasso  = predict_centered(Xte, theta_lasso, y_mean)
add_result("Lasso scikit", theta_lasso, yhat_lasso, t0)

# ---- Print leaderboard (sorted by MSE) ----
results_sorted = sorted(results, key=lambda d: d["MSE"])
w = max(len(r["method"]) for r in results_sorted)
print("\n" + "="*(w+44))
print(" LEADERBOARD (lower MSE is better) ")
print("="*(w+44))
print(f"{'method'.ljust(w)}   MSE          R²        ||θ||₂       time[s]")
for r in results_sorted:
    print(f"{r['method'].ljust(w)}   {r['MSE']:.6f}   {r['R2']:.4f}   {r['||θ||2']:.3e}   {r['time_s']:.3f}")

best = results_sorted[0]
print("\nBest by MSE:", best["method"], f"(MSE={best['MSE']:.6f}, R²={best['R2']:.4f})")



 LEADERBOARD (lower MSE is better) 
method                        MSE          R²        ||θ||₂       time[s]
Ridge closed-form             0.127581   0.3595   1.522e+01   0.001
OLS adam                      0.129212   0.3513   5.359e+00   0.133
Ridge adam                    0.130551   0.3446   4.286e+00   0.113
OLS closed-form               0.135149   0.3215   1.771e+03   0.005
OLS rmsprop                   0.135171   0.3214   3.005e+00   0.076
Lasso scikit                  0.135397   0.3202   2.240e+00   0.007
Ridge rmsprop                 0.135495   0.3197   2.890e+00   0.086
OLS gd                        0.150165   0.2461   5.149e-01   0.093
OLS momentum                  0.150165   0.2461   5.149e-01   0.057
Ridge gd                      0.150167   0.2461   5.145e-01   0.057
Ridge momentum                0.150167   0.2461   5.145e-01   0.069
OLS adagrad                   0.150755   0.2431   6.355e-01   0.061
Ridge adagrad                 0.150765   0.2431   6.310e-01   0.074
OLS r